In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import random
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt
import math 
import random
import miceforest as mf
import sys 
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)


import gc

In [ ]:
data = pd.read_table('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data.txt',delimiter = '|') 

In [ ]:
data.head(4) 

In [ ]:
data[data.Estacion == "aranjuez"].Precipitacion.plot() 

In [ ]:
datas = data[["Fecha","Hora","Temperatura","Velocidad","Presion","Humedad","Precipitacion","Radiacion"]]
datas.head(3) 

In [ ]:
corr_matrix = datas.corr(method='pearson') 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10)) 

sns.heatmap(
    corr_matrix,
    annot     = True,
    cbar      = False,
    annot_kws = {"size": 8},
    vmin      = -1,
    vmax      = 1,
    center    = 0,
    cmap      = sns.diverging_palette(20, 220, n=200),
    square    = True,
    ax        = ax
)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)

ax.tick_params(labelsize = 10)

In [ ]:
corr_matrix["Precipitacion"].sort_values(ascending=False) 

In [ ]:
import missingno as msno  
%matplotlib inline
msno.matrix(datas,figsize=(15,6),fontsize=12)  

In [ ]:
datas = datas[["Fecha","Temperatura","Velocidad","Presion","Humedad","Precipitacion", "Radiacion"]]
datas.info()  

In [ ]:
datas["Fecha"] = pd.to_datetime(datas["Fecha"])  

In [ ]:
datas["year"] = datas["Fecha"].dt.year  
datas["month"] = datas["Fecha"].dt.month 
datas["day"] = datas["Fecha"].dt.day  

In [ ]:
datas = datas.drop(columns={"Fecha"}, axis = 1) 

In [ ]:
datas

Technically, any predictive model capable of inference can be used for MICE. In this article, we impute a dataset with the miceforest Python library, which uses lightgbm random forests by default (although this can be changed). Random forests work well with the MICE algorithm for several reasons: <br/>

1. Do not need much hyperparameter tuning
2. Easily handle non-linear relationships in the data
3. Can return OOB performance inexpensively
4. Are trivially parallelizable
5. Can return feature importance for diagnostics

In [ ]:
# Create kernel. 
kds = mf.ImputationKernel(
  datas,  
  #datasets=3, 
  save_all_iterations=True,
  random_state=36
)

# Run the MICE algorithm for 2 iterations
kds.mice(iterations=5)

# Return the completed dataset.
complete = kds.complete_data(0)
print(complete.isnull().sum())

**What we have done is created 5 separate datasets with different imputed values.** <br/>
**We can never be sure what the original data was, but if our different datasets all come up with similar imputed values, we can say that we are confident in our imputations. Let’s take a look at the correlations of the imputed values between datasets:**

In [ ]:
kds.plot_correlations(wspace=.5,hspace=2)

In [ ]:
kds.plot_imputed_distributions(wspace=0.35,hspace=0.4)

In [ ]:
complete.Precipitacion.plot()

In [ ]:
#plt.figure(figsize=(12,8))
datas.Precipitacion.plot()   

## MICE 

In [ ]:
# Create kernel. 
kds_2 = mf.ImputationKernel(
  datas,  
  datasets=1, 
  save_all_iterations=True,
  random_state=36
)

# Run the MICE algorithm for 2 iterations
kds_2.mice(iterations=5)    

# Return the completed dataset.
complete_2 = kds_2.complete_data(0)
print(complete_2.isnull().sum())

In [ ]:
#plt.figure(figsize=(8,6))
complete_2.Precipitacion.plot()  

In [ ]:
#plt.figure(figsize=(8,6))
datas.Precipitacion.plot()    

In [ ]:
complete_2.Temperatura.plot()

In [ ]:
datas.Temperatura.plot()     

In [ ]:
complete_2

In [ ]:
complete_2.to_csv('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data_imputed_MICE.txt', sep = '|', index=False, header=True)

In [ ]:
complete_2.insert(loc =6,column='Estacion', value = data.Estacion) 

In [ ]:
tipos = complete_2.Estacion.unique() 
tipos 

In [ ]:
for i in tipos:
    print(i) 
    complete_2[complete_2.Estacion == i].Precipitacion.plot() 
    plt.show()

## MICE con nuevas variables 

In [ ]:
data["Fecha"] = pd.to_datetime(data["Fecha"])   
datas["year"] = data["Fecha"].dt.year  
datas["month"] = data["Fecha"].dt.month 
datas["day"] = data["Fecha"].dt.day  

In [ ]:
#datas = datas.drop(columns={"Fecha"}, axis = 1) 

In [ ]:
# Create kernel. 
kds_3 = mf.ImputationKernel(
  datas,  
  datasets=1, 
  save_all_iterations=True,
  random_state=36
)

# Run the MICE algorithm for 2 iterations
kds_3.mice(iterations=5)    

# Return the completed dataset.
complete_3 = kds_3.complete_data(0)
print(complete_3.isnull().sum())

In [ ]:
complete_3.Precipitacion.plot()  

In [ ]:
datas.Precipitacion.plot()    

In [ ]:
complete_3.insert(loc =9,column='Estacion', value = data.Estacion)  

In [ ]:
for i in tipos:
    print(i) 
    complete_3[complete_3.Estacion == i].Precipitacion.plot()  
    plt.show()

In [ ]:
complete_3.to_csv('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data_imputed_MICE_3.txt', sep = '|', index=False, header=True)

In [ ]:
plt.figure(figsize=(12,8))
complete_2[complete_2.Precipitacion > 200].Precipitacion.plot()

In [ ]:
dt["Precipitacion"] = np.where((dt.Precipitacion >300 ), 250, dt["Precipitacion"])  

## random forest


In [ ]:
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [ ]:
from missingpy import MissForest  

In [ ]:
datas 

In [ ]:
imputer = MissForest()

data_imputed = imputer.fit_transform(datas)

In [ ]:
df_imputed = pd.DataFrame(data_imputed) 

In [ ]:
col_list = tuple(datas.columns.values) 

In [ ]:
old_col_names = tuple(df_imputed.columns.values)

In [ ]:
df_imputed.rename(
    columns={i:j for i,j in zip(old_col_names,col_list)}, inplace=True
) 

In [ ]:
df_imputed

In [ ]:
df_imputed.Precipitacion.plot() 

In [ ]:
datas.Precipitacion.plot()  

In [ ]:
df_imputed

In [ ]:
#df_imputed.to_csv('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data_imputed_rf.txt', sep = '|', index=False, header=True)

In [ ]:
df_imputed.insert(loc =9,column='Estacion', value = data.Estacion)

In [ ]:
df_imputed

In [ ]:
for i in tipos:
    print(i) 
    df_imputed[df_imputed.Estacion == i].Precipitacion.plot()   
    plt.show()

In [ ]:
df_imputed.Temperatura.plot()  

In [ ]:
datas.Temperatura.plot()  

In [ ]:
df_imputed.Humedad.plot()  

In [ ]:
datas.Humedad.plot()   

## knn Imputer

In [ ]:
#imputer = KNNImputer(n_neighbors=5)
#imputer_fit = imputer.fit_transform(datas)

In [ ]:
knn_imputed = pd.DataFrame(imputer_fit, columns=datas.columns)

In [ ]:
knn_imputed.to_csv('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data_imputed_knn.txt', sep = '|', index=False, header=True)

## Cargamos los datos nuevamente

In [ ]:
complete_2 = pd.read_table('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data_imputed_MICE.txt',delimiter = '|') 

In [ ]:
df_imputed = pd.read_table('C:/Users/pulzara/Documents/Universidad internacional de valencia/TFM/base de datos/SIMAC organizados/BD2/data_imputed_rf.txt',delimiter = '|') 

In [ ]:
df_imputed 

In [ ]:
complete_2 

In [ ]:
#complete_2.insert(loc = 0,column="Fecha", value= data.Fecha) 
#df_imputed.insert(loc = 0,column="Fecha", value= data.Fecha) 
#df_imputed.insert(loc = 1,column="Hora", value= data.Hora) 
#complete_2.insert(loc = 1,column="Hora", value= data.Hora)
#complete_2.insert(loc = 8,column="Estacion", value= data.Estacion)  
#df_imputed.insert(loc = 8,column="Estacion", value= data.Estacion)  

In [ ]:
df_imputed["year"] = df_imputed.year.astype(int)
df_imputed["month"] = df_imputed.month.astype(int) 
df_imputed["day"] = df_imputed.day.astype(int)

In [ ]:
complete_2["year"] = complete_2.year.astype(int)
complete_2["month"] = complete_2.month.astype(int)
complete_2["day"] = complete_2.day.astype(int) 

In [ ]:
df_imputed.info()

In [ ]:
df_imputed

In [ ]:
df_imputed.set_index("Fecha",drop=True, inplace=True)  

## Random forest

In [ ]:
x = df_imputed[["Temperatura", "Velocidad", "Presion", "Humedad", "Radiacion"]]
y = df_imputed["Precipitacion"]

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(x) 

In [ ]:
rf = RandomForestRegressor(n_estimators = 80, random_state = 40)
rf_fit = rf.fit(X, y)   

In [ ]:
rf_fit.feature_importances_

In [ ]:
importances = rf_fit.feature_importances_
columns = x.columns 
indices = np.argsort(importances)[::-1]

plt.figure(figsize = (12,6))
sns.barplot(columns[indices], importances[indices])
plt.xticks(rotation=85)
plt.show

### PCA 

In [ ]:
df_pca = df_imputed[['Temperatura','Velocidad','Presion','Humedad','Precipitacion','Radiacion']]

In [ ]:
pca = PCA() 
pca.fit(df_pca)  

### dataset2

In [ ]:
x_1 = complete_2[["Temperatura", "Velocidad", "Presion", "Humedad", "Radiacion"]]
y_1 = complete_2["Precipitacion"] 

In [ ]:
scaler2 = StandardScaler()
X2 = scaler2.fit_transform(x_1) 

In [ ]:
rf2 = RandomForestRegressor(n_estimators = 80, random_state = 40)
rf_fit2 = rf2.fit(X2, y_1)   

In [ ]:
rf_fit2.feature_importances_

In [ ]:
importances = rf_fit2.feature_importances_
columns = x_1.columns 
indices = np.argsort(importances)[::-1]

plt.figure(figsize = (12,6))
sns.barplot(columns[indices], importances[indices])
plt.xticks(rotation=85)
plt.show

## random forest - forecasterAutoreg 

In [ ]:
def train_test_split(data, fecha):
    
    test = data[data.index <= fecha]
    train = data[data.index > fecha]
    return test, train

In [ ]:
train, test = train_test_split(df_imputed,"2020-12-25")

In [ ]:
train 

In [ ]:
plt.figure(figsize = (12,8))
plt.plot(train.Precipitacion,label='train')
plt.plot(test.Precipitacion, label= 'test')  
plt.legend(loc = "upper left")
plt.show 